In [ ]:
 #link dataset : https://drive.google.com/file/d/1-w9tMvlswf1PIxygb7KJ1Bbwy4UI45rx/view?usp=sharing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp /content/drive/MyDrive/ML_ObjectDetectionModelSmall.zip /content/

In [ ]:
import zipfile

with zipfile.ZipFile("/content/ML_ObjectDetectionModelSmall.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/dataset")


In [ ]:
pip install keras-cv


In [ ]:
import os
import tensorflow as tf
import numpy as np
import random
from tensorflow import keras
import keras_cv
from sklearn.model_selection import train_test_split

In [ ]:
import os
import tensorflow as tf

# 1. Definisikan Direktori Dataset
image_dir = '/content/dataset/ML_ObjectDetectionModelSmall/data'
train_images_dir = os.path.join(image_dir, 'train')
val_images_dir = os.path.join(image_dir, 'validation')

# 2. Fungsi untuk memeriksa apakah gambar valid
def is_valid_image(image_path):
    try:
        image = tf.io.read_file(image_path)
        tf.image.decode_jpeg(image)  # Ganti dengan decode sesuai format jika perlu
        return True
    except Exception as e:
        print(f"File {image_path} tidak valid: {e}")
        return False

# 3. Fungsi untuk mengonversi gambar ke JPEG
def convert_image_to_jpeg(image_path, target_size=(416, 416)):
    # Membaca file gambar
    image = tf.io.read_file(image_path)

    # Menentukan format gambar asli
    image_format = tf.image.decode_image(image, channels=3, dtype=tf.dtypes.uint8)

    # Jika gambar hanya memiliki 1 channel (grayscale), konversi ke RGB
    if image_format.shape[-1] == 1:
        image_format = tf.image.grayscale_to_rgb(image_format)

    # Mengonversi gambar ke JPEG setelah resize
    image_resized = tf.image.resize(image_format, target_size) / 255.0  # Normalisasi
    image_jpeg = tf.image.encode_jpeg(tf.cast(image_resized * 255.0, tf.uint8))  # Encode ke JPEG

    return image_jpeg

# 4. Fungsi untuk menyimpan gambar hasil konversi ke file JPEG
def save_converted_image(image_jpeg, output_path):
    tf.io.write_file(output_path, image_jpeg)

# 5. Fungsi untuk mengonversi semua gambar di direktori ke format JPEG
def convert_all_images_to_jpeg(image_dir, target_dir, target_size=(416, 416)):
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    # Iterasi semua gambar dalam direktori
    for subdir, dirs, files in os.walk(image_dir):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp')):  # Semua format gambar
                image_path = os.path.join(subdir, file)

                # Cek apakah gambar valid
                if is_valid_image(image_path):
                    # Menentukan nama file output (ganti ekstensi dengan .jpg)
                    output_filename = file.split('.')[0] + '.jpg'

                    # Membuat subfolder yang sama di target_dir
                    relative_subdir = os.path.relpath(subdir, image_dir)
                    output_subdir = os.path.join(target_dir, relative_subdir)

                    if not os.path.exists(output_subdir):
                        os.makedirs(output_subdir)  # Membuat subfolder jika belum ada

                    output_path = os.path.join(output_subdir, output_filename)

                    # Konversi gambar dan simpan
                    image_jpeg = convert_image_to_jpeg(image_path, target_size)
                    save_converted_image(image_jpeg, output_path)
                    print(f"Berhasil mengonversi {image_path} ke {output_path}")
                else:
                    print(f"{image_path} tidak valid, tidak diproses.")

# 6. Menjalankan fungsi untuk folder 'train' dan 'validation'
convert_all_images_to_jpeg(train_images_dir, '/content/dataset/ML_ObjectDetectionModelSmall/data_augmented/train')
convert_all_images_to_jpeg(val_images_dir, '/content/dataset/ML_ObjectDetectionModelSmall/data_augmented/val')


Streaming output truncated to the last 5000 lines.
Berhasil mengonversi /content/dataset/ML_ObjectDetectionModelSmall/data/train/kemiri/kemiri_72.JPG ke /content/dataset/ML_ObjectDetectionModelSmall/data_augmented/train/kemiri/kemiri_72.jpg
Berhasil mengonversi /content/dataset/ML_ObjectDetectionModelSmall/data/train/kemiri/kemiri_178.JPG ke /content/dataset/ML_ObjectDetectionModelSmall/data_augmented/train/kemiri/kemiri_178.jpg
Berhasil mengonversi /content/dataset/ML_ObjectDetectionModelSmall/data/train/kemiri/kemiri_146.JPG ke /content/dataset/ML_ObjectDetectionModelSmall/data_augmented/train/kemiri/kemiri_146.jpg
Berhasil mengonversi /content/dataset/ML_ObjectDetectionModelSmall/data/train/kemiri/kemiri_164.jpg ke /content/dataset/ML_ObjectDetectionModelSmall/data_augmented/train/kemiri/kemiri_164.jpg
Berhasil mengonversi /content/dataset/ML_ObjectDetectionModelSmall/data/train/kemiri/kemiri_29.JPG ke /content/dataset/ML_ObjectDetectionModelSmall/data_augmented/train/kemiri/kemiri_

In [ ]:
augmented_train_dir = '/content/dataset/ML_ObjectDetectionModelSmall/data_augmented/train'
augmented_val_dir = '/content/dataset/ML_ObjectDetectionModelSmall/data_augmented/val'
Tf_record_dir = '/content/dataset/ML_ObjectDetectionModelSmall/tf_record'

In [ ]:
def create_yolo_labels(image_path, class_id, image_size=(416, 416)):
    """
    Membuat anotasi YOLO berdasarkan ukuran gambar penuh (bounding box penuh).
    """
    # Membaca gambar dan mengubah ukurannya
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, image_size) / 255.0  # Normalisasi ke [0, 1]

    # Tentukan bounding box penuh
    h, w, _ = image.shape
    x_center = 0.5
    y_center = 0.5
    width = 1.0
    height = 1.0

    # Format YOLO: [class_id, x_center, y_center, width, height]
    label = np.array([class_id, x_center, y_center, width, height], dtype=np.float32)

    return image, label

In [ ]:
def image_to_tfrecord(image_path, label, image_size=(416, 416)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, image_size)  # Resize gambar
    image = tf.io.encode_jpeg(image)  # Encode gambar menjadi JPEG

    feature = {
        'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image.numpy()])),  # Gambar
        'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))  # Label
    }

    example = tf.train.Example(features=tf.train.Features(feature=feature))
    return example

In [ ]:
def augment_data(image, label):
    # Contoh augmentasi
    image = tf.image.random_flip_left_right(image)  # Flip horizontal
    image = tf.image.random_flip_up_down(image)  # Flip vertikal
    image = tf.image.random_contrast(image, lower=0.2, upper=1.8)  # Kontras acak
    image = tf.image.random_brightness(image, max_delta=0.2)  # Kecerahan acak

    # Anda bisa menambahkan augmentasi lainnya seperti rotasi, zoom, dll.

    return image, label

In [ ]:
def write_tfrecord(image_paths, labels, output_path):
    # Menulis file TFRecord
    with tf.io.TFRecordWriter(output_path) as writer:
        for img_path, label in zip(image_paths, labels):
            # Membaca gambar
            image = tf.io.read_file(img_path)
            image = tf.image.decode_jpeg(image, channels=3)  # Decode gambar JPG

            # Normalisasi gambar ke [0, 255] dan ubah tipe data menjadi uint8
            image = tf.image.convert_image_dtype(image, dtype=tf.uint8)

            # Membuat fitur untuk gambar dan label
            feature = {
                'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.io.encode_jpeg(image).numpy()])),
                'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))
            }

            # Membuat contoh (Example) dan menulisnya ke TFRecord
            example = tf.train.Example(features=tf.train.Features(feature=feature))
            writer.write(example.SerializeToString())


In [ ]:
def is_image_file(file_path):
    try:
        # Membaca gambar untuk memastikan file tersebut valid
        image = tf.io.read_file(file_path)
        image = tf.image.decode_jpeg(image, channels=3)  # Coba decode sebagai JPEG
        return True  # Jika berhasil, file adalah gambar yang valid
    except tf.errors.InvalidArgumentError:
        return False  # Jika gagal decode, file bukan gambar atau format rusak


In [ ]:
def prepare_and_convert_to_tfrecord(image_dir, output_dir, image_size=(416, 416)):
    class_names = sorted(os.listdir(image_dir))  # Daftar nama subfolder yang jadi kelas
    image_paths = []
    labels = []

    for class_id, class_name in enumerate(class_names):
        class_folder = os.path.join(image_dir, class_name)
        if os.path.isdir(class_folder):
            for img_name in os.listdir(class_folder):
                img_path = os.path.join(class_folder, img_name)
                if is_image_file(img_path):  # Hanya tambahkan file gambar yang valid
                    image_paths.append(img_path)
                    labels.append(class_id)  # Label sesuai dengan urutan subfolder

    # Tentukan file output TFRecord
    output_path = os.path.join(output_dir, 'dataset.tfrecord')
    write_tfrecord(image_paths, labels, output_path)

    # Kembalikan paths, labels, dan class_names
    return image_paths, labels, class_names


In [ ]:
def _parse_function(proto):
    keys_to_features = {
        'image': tf.io.FixedLenFeature([], tf.string),  # Gambar dalam format string
        'label': tf.io.FixedLenFeature([1], tf.int64),  # Label sebagai integer
    }

    parsed_features = tf.io.parse_single_example(proto, keys_to_features)

    # Decode gambar
    image = tf.io.decode_jpeg(parsed_features['image'])
    label = parsed_features['label'][0]

    # Resizing gambar ke ukuran yang diinginkan
    image = tf.image.resize(image, [416, 416])  # Sesuaikan dengan ukuran input model Anda

    return image, label


In [ ]:
def load_tfrecord_dataset(tfrecord_file, batch_size=16, augment=False):
    # Memuat dataset dari file TFRecord
    dataset = tf.data.TFRecordDataset([tfrecord_file])

    # Parsing data
    dataset = dataset.map(_parse_function)

    if augment:
        # Terapkan augmentasi jika diperlukan
        dataset = dataset.map(augment_data)

    # Shuffle, batching, dan prefetch untuk performa yang lebih baik
    dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)  # Prefetch untuk mempercepat pemrosesan
    return dataset

In [ ]:
#hapus dlu folder anotasi(disemua subfolder)

In [ ]:
train_image_paths, train_labels, train_class_names = prepare_and_convert_to_tfrecord(train_images_dir, Tf_record_dir)
val_image_paths, val_labels, val_class_names = prepare_and_convert_to_tfrecord(val_images_dir, Tf_record_dir)

In [ ]:


# 7. Memuat dataset dari file TFRecord untuk train dan validation
train_tfrecord_file = os.path.join(Tf_record_dir, 'train.tfrecord')
val_tfrecord_file = os.path.join(Tf_record_dir, 'validation.tfrecord')

train_dataset = load_tfrecord_dataset(train_tfrecord_file, batch_size=16, augment=True)  # Terapkan augmentasi pada data training
val_dataset = load_tfrecord_dataset(val_tfrecord_file, batch_size=16, augment=False)  # Tidak perlu augmentasi pada data validation


In [ ]:
backbone = keras_cv.models.YOLOV8Backbone.from_preset(
    "yolo_v8_s_backbone_coco"  # We will use yolov8 small backbone with coco weights
)

In [ ]:
model = keras_cv.models.YOLOV8Detector(
    bounding_box_format="xywh",  # Format YOLO
    num_classes=len(train_class_names),  # Jumlah kelas yang ditemukan
    backbone=backbone,  # Gunakan backbone default dari YOLOv8
)

In [ ]:
# Cek output model
print(model.output)


{'boxes': <KerasTensor shape=(None, None, 64), dtype=float32, sparse=False, name=keras_tensor_1038>, 'classes': <KerasTensor shape=(None, None, 30), dtype=float32, sparse=False, name=keras_tensor_1039>}


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Custom layer to expand dimensions
class ExpandDimsLayer(layers.Layer):
    def __init__(self, axis=1):
        super(ExpandDimsLayer, self).__init__()
        self.axis = axis

    def call(self, inputs):
        return tf.expand_dims(inputs, axis=self.axis)

# Ambil output kelas dari model
classes_output = model.output['classes']

# Terapkan custom layer untuk menambahkan dimensi
x = ExpandDimsLayer(axis=1)(classes_output)  # Menambahkan dimensi height (None, 1, None, 3)

# Terapkan GlobalAveragePooling2D ke output kelas
x = tf.keras.layers.GlobalAveragePooling2D()(x)

# Menambahkan dense layer
x = tf.keras.layers.Dense(512, activation="relu")(x)  # Dense layer pertama
x = tf.keras.layers.Dense(256, activation="relu")(x)  # Dense layer kedua

# Output layer
output = tf.keras.layers.Dense(len(train_class_names), activation="softmax")(x)

# Membuat model baru dengan input dari model lama dan output dari layer Dense yang ditambahkan
custom_model = tf.keras.Model(inputs=model.input, outputs=output)

# Kompilasi model
custom_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Summary dari model untuk memeriksa arsitektur
custom_model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5             │ (None, None, None, 3)  │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_2 (Functional) │ [(None, None, None,    │      5,089,760 │ input_layer_5[0][0]    │
│                           │ 128), (None, None,     │                │                        │
│                           │ None, 256), (None,     │                │                        │
│                           │ None, None, 512)]      │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ repeat_4 (Repeat)         │ (None, None, None,     │              0 │ functional_2[0][2]     │
│                           │ 512)                   │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ repeat_5 (Repeat)         │ (None, None, None,     │              0 │ repeat_4[0][0]         │
│                           │ 512)                   │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_22            │ (None, None, None,     │              0 │ repeat_5[0][0],        │
│ (Concatenate)             │ 768)                   │                │ functional_2[0][1]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pa_fpn_p4p5_pre_conv      │ (None, None, None,     │        196,608 │ concatenate_22[0][0]   │
│ (Conv2D)                  │ 256)                   │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pa_fpn_p4p5_pre_bn        │ (None, None, None,     │          1,024 │ pa_fpn_p4p5_pre_conv[… │
│ (BatchNormalization)      │ 256)                   │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pa_fpn_p4p5_pre           │ (None, None, None,     │              0 │ pa_fpn_p4p5_pre_bn[0]… │
│ (Activation)              │ 256)                   │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ split_12 (Split)          │ [(None, None, None,    │              0 │ pa_fpn_p4p5_pre[0][0]  │
│                           │ 128), (None, None,     │                │                        │
│                           │ None, 128)]            │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pa_fpn_p4p5_pre_0_1_pad   │ (None, None, None,     │              0 │ split_12[0][1]         │
│ (ZeroPadding2D)           │ 128)                   │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pa_fpn_p4p5_pre_0_1_conv  │ (None, None, None,     │        147,456 │ pa_fpn_p4p5_pre_0_1_p… │
│ (Conv2D)                  │ 128)                   │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pa_fpn_p4p5_pre_0_1_bn    │ (None, None, None,     │            512 │ pa_fpn_p4p5_pre_0_1_c… │
│ (BatchNormalization)      │ 128)                   │                │                        │
├──────────────────────

 Total params: 13,374,744 (51.02 MB)

 Trainable params: 13,352,408 (50.94 MB)

 Non-trainable params: 22,336 (87.25 KB)

In [ ]:
custom_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss="sparse_categorical_crossentropy",  # Loss untuk klasifikasi
    metrics=["accuracy"]
)

In [ ]:
import os

# Periksa apakah file TFRecord ada
print(os.listdir(Tf_record_dir))



['dataset.tfrecord']


In [ ]:
custom_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    verbose=1
)

Epoch 1/10


NotFoundError: Graph execution error:

Detected at node IteratorGetNext defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/python3.10/dist-pack

In [ ]:
model.save("yolo_rasadahana.h5")

# Simpan model ke dalam format JSON
model_json = model.to_json()
with open("yolo_rasadahana.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
def get_predicted_labels(image, model, class_names):
    """
    Menghasilkan label kelas berdasarkan hasil prediksi dan bounding box.
    """
    predictions = model.predict(tf.expand_dims(image, axis=0))  # Prediksi untuk 1 gambar
    pred_boxes, pred_scores, pred_classes = predictions[0]

    predicted_labels = []
    for i in range(len(pred_boxes)):
        if pred_scores[i] > 0.5:  # Ambil hanya prediksi dengan score > threshold
            class_id = int(pred_classes[i])
            predicted_labels.append(class_names[class_id])

    return predicted_labels

In [ ]:
test_image_path = 'path_to_test_image.jpg'  # Ganti dengan path gambar uji
test_image = tf.io.read_file(test_image_path)
test_image = tf.image.decode_jpeg(test_image, channels=3)
test_image = tf.image.resize(test_image, (416, 416)) / 255.0

# Menggunakan model untuk mendapatkan label dari gambar uji
predicted_labels = get_predicted_labels(test_image, model, class_names)
print(f"Predicted labels: {predicted_labels}")